# HUHU@IberLEF2023 Task 3 (Regression)

Task: https://sites.google.com/view/huhuatiberlef23/huhu

This notebook contains the code to load and test several trained transformers for the task of hurtful humour detection (regression).

In particular, the models are:

* BERT Multilingual: ``bert-base-multilingual-cased`` and ``bert-base-multilingual-uncased``
* RoBERTa: ``roberta-base``
* BETO: ``dccuchile/bert-base-spanish-wwm-cased`` and ``dccuchile/bert-base-spanish-wwm-uncased``
* DistilBERT Multilingual: ``distilbert-base-multilingual-cased``

The transformers that define the best ensemble found in experimentation will be used to predict the score of the instances in the test split.

The predicted scores will be stored in an output file as per the requirements of the competition. Additional evaluation plots may be also genered.

# Setting up the environment

In [ ]:
import torch

# Check GPU availability on Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

use_cuda = torch.cuda.is_available()

/bin/bash: nvidia-smi: command not found


In [ ]:
# Install libraries
!pip install simpletransformers
!pip install datasets
!pip install ipywidgets
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:
# Define global variables

SEED = 42 # allow for experiments' reproductibility
WEIGHTED = True # use weighted ensemble (in favour of models with higher F1-score)

# Test split load

In [ ]:
from huggingface_hub import notebook_login
# Notebook login via HF's token
notebook_login()

In [ ]:
from datasets import *
import pandas as pd

# Avoid warnings
logging.set_verbosity_error()

# Load test split
test = pd.DataFrame(load_dataset("huhu2023/test-huhu2023", split="test"))

In [ ]:
# Function to rename fields and drop unnecessary ones
def get_text_and_label(df, original_dataset=True):
  return df.rename(columns={"tweet": "text"})[["index", "text"]]

# Get treated dataframe for test split
test = get_text_and_label(test)

print("Test split size:", len(test.index))
test.head()

Test split size: 778


,index,text
0,52830,-Mamá en la escuela me dicen gorda -Pobresilla...
1,78883,"No te sientas diferente, da igual si eres negr..."
2,78926,Si esta asi.. SUPER SI.. y que se pongan celos...
3,61844,—Bebé ¿Me veo gorda con este vestido?\n—¡No mi...
4,78830,Las mujeres solo desean 2 cosas en la vida: co...


# Set-up the working environment

In [ ]:
# Select the name of the experiment to be evaluated
EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
TRANSFORMERS = "top2_transformers"

In [ ]:
# Load and mount the Drive helper
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Define the path to the experiment folder
PATH = "/content/drive/My Drive/HUHU-IberLEF2023/reg/outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)

# Create a folder for the test results
OUTPUT = os.path.join(EXP_PATH, "test")
os.mkdir(OUTPUT)

Current working dir: /content/drive/My Drive/HUHU-IberLEF2023/reg/outputs/lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining


# Models' load

In this section, the different transformers that will be tested are gathered. For this purpose, the implementation mainly relies in the ``simpletransformers`` Python library, which allows to train and test transformers within few steps.

For further information: https://simpletransformers.ai/

In [ ]:
# Define transformers' initialization dictionary 
models = {
    "mbert-cased": {
        "model_type": "bert",
        "model_name": "bert-base-multilingual-cased"
    },
    "mbert-uncased": {
        "model_type": "bert",
        "model_name": "bert-base-multilingual-uncased"
    },
    "roberta": {
        "model_type": "roberta",
        "model_name": "roberta-base"
    },
    "beto-cased": {
        "model_type": "bert",
        "model_name": "dccuchile/bert-base-spanish-wwm-cased"
    },
    "beto-uncased": {
        "model_type": "bert",
        "model_name": "dccuchile/bert-base-spanish-wwm-uncased"
    },
    "distilbert-multi": {
        "model_type": "distilbert",
        "model_name": "distilbert-base-multilingual-cased"
    }
}

In [ ]:
# Import pre-trained simpletransformers models for classification
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# Define a dictionary where each key matches its corresponding transformer
for model, fields in models.items():    
  models[model] = ClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model))

# Best ensemble's definition

The ensemble that performed the best in the selected experiment is defined.

This will be the one used for the predictions to be performed on the test set.

In [ ]:
import json

# Get the data relative to the best ensemble
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)

print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble40
models: ['beto-cased', 'beto-uncased', 'distilbert-multi']
metrics: {'r2_score': 0.3484303, 'mae': 0.5042991, 'mse': 0.414243, 'rmse': 0.6436171}


# Best ensemble's predictions

In [ ]:
# Load model evaluation JSON
model_evaluation = {}
for model in best_ensemble.get("models"):
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [ ]:
from sklearn.preprocessing import normalize

# Function which determines the ensembler prediction based on its
# transformers' predictions. A weighted voting system may be used
def vote(predictions, weighted=False, weights=None):
  return sum(predictions * weights) if weighted else sum(predictions)/len(predictions)

test_predictions = list()

# Function to predict the label of the instances in a dataset split (validation
# ("val") or test ("test")) for each ensemble
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):
  # Traverse each dataset instance
  for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")
    # Get the raw output of each model in the ensemble for the instance at hand
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_scores")
      predictions.append(curr_model_outputs[i])
    
    # Define the list of weights if a weighted voting system must be used
    weights = list()
    if weighted:
      # The weights' list is obtained by normalizing the RMSE of the models
      # in the ensemble
      rmse_list = [model_evaluation[model_name]["metrics"].get("rmse")
                        for model_name in best_ensemble.get("models")]
      weights = normalize([[1/rmse for rmse in rmse_list]], norm="l1")[0]

    # Append the computed scores to the predictions of the ensemble
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [ ]:
# Predicting the score of the test set's instances with each individual
# transformer that composes the best ensemble
for model_name in best_ensemble.get("models"):
  _, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_scores"] = model_raw_outputs

# Calculating the test predictions of the best ensemble
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)

# Show some predictions
n = 5
print(f"First {n} predictions:", test_predictions[:n])

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

First 5 predictions: [3.0797690761643812, 3.913602156800313, 2.9057258882841133, 2.197828404376431, 3.410843044049252]


# Save test predictions

A new pandas dataframe is created. Further, the test predictions are saved to an output CSV file as per required by the competition.

In [ ]:
# Create output dataframe for test predictions
test_output = test[["index"]].rename(columns={"index": "tweet_id"})
# Rounding to only one decimal value
test_predictions = [round(pred, 1) for pred in test_predictions]
test_output["prejudice_degree"] = test_predictions
test_output.head(10)

,tweet_id,prejudice_degree
0,52830,3.1
1,78883,3.9
2,78926,2.9
3,61844,2.2
4,78830,3.4
5,48393,2.8
6,11875,3.1
7,62245,2.8
8,62467,3.8
9,40664,2.6


In [ ]:
# Save results in CSV file
test_output.to_csv(os.path.join(OUTPUT, "results.csv"), index=False)